In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd

# ---------------------------------------------------
# Page setup
# ---------------------------------------------------
# Remove this line if st.set_page_config is already called earlier in your app.
# st.set_page_config(layout="wide")

st.title("YouTube Channels – Gold Layer Dashboard")
st.caption(
    "Built on DB_TEAM_VRR.GOLD tables: "
    "CHANNEL_GROWTH_METRICS, CHANNEL_EFFICIENCY_LEADERBOARD, COUNTRY_PORTFOLIO_SHARE"
)

session = get_active_session()

# ---------------------------------------------------
# Helper: cached loader + refresh
# ---------------------------------------------------
def load_or_cache(query: str, key: str) -> pd.DataFrame:
    if key not in st.session_state:
        df = session.sql(query).to_pandas()
        st.session_state[key] = df
    return st.session_state[key]

# Refresh button to clear cached dataframes
if st.button("🔄 Refresh data from Snowflake"):
    for k in ["growth_df", "eff_df", "port_df"]:
        st.session_state.pop(k, None)
    st.success("Data cache cleared. Latest data will be loaded from Snowflake on this run.")

# ---------------------------------------------------
# Load Gold data
# ---------------------------------------------------
growth_query = """
SELECT *
FROM DB_TEAM_VRR.GOLD.CHANNEL_GROWTH_METRICS;
"""

eff_query = """
SELECT *
FROM DB_TEAM_VRR.GOLD.CHANNEL_EFFICIENCY_LEADERBOARD;
"""

portfolio_query = """
SELECT *
FROM DB_TEAM_VRR.GOLD.COUNTRY_PORTFOLIO_SHARE;
"""

growth_df = load_or_cache(growth_query, "growth_df")
eff_df = load_or_cache(eff_query, "eff_df")
port_df = load_or_cache(portfolio_query, "port_df")

# Normalize LOAD_DATE as full datetime (keep time component)
for df in (growth_df, port_df):
    if "LOAD_DATE" in df.columns:
        df["LOAD_DATE"] = pd.to_datetime(df["LOAD_DATE"])

st.markdown("---")

# ===================================================
# 1) Top Channels by Subscribers (Bar chart)
# ===================================================

st.subheader("1. Top Channels by Subscribers (by Load Date / Time)")

col1, col2, col3 = st.columns(3)

# Load date/time filter
with col1:
    available_loads = sorted(growth_df["LOAD_DATE"].dropna().unique())
    if len(available_loads) == 0:
        selected_load = None
        st.warning("No LOAD_DATE values found in growth data.")
    else:
        selected_load = st.selectbox(
            "Load Date / Time",
            options=available_loads,
            index=len(available_loads) - 1,
            key="growth_load_datetime",
            format_func=lambda x: x.strftime("%Y-%m-%d %H:%M:%S"),
        )

# Category filter
with col2:
    categories = sorted(growth_df["CATEGORY_NAME"].dropna().unique().tolist())
    selected_category = st.selectbox(
        "Category (optional)",
        options=["All"] + categories,
        key="growth_category",
    )

# Country filter
with col3:
    countries = sorted(growth_df["COUNTRY_CODE"].dropna().unique().tolist())
    selected_country = st.selectbox(
        "Country (optional)",
        options=["All"] + countries,
        key="growth_country",
    )

if selected_load is not None:
    g = growth_df[growth_df["LOAD_DATE"] == selected_load].copy()

    if selected_category != "All":
        g = g[g["CATEGORY_NAME"] == selected_category]

    if selected_country != "All":
        g = g[g["COUNTRY_CODE"] == selected_country]

    # Sort by subscriber count and take top 10
    g = g.sort_values("SUBSCRIBER_COUNT", ascending=False).head(10)

    if g.empty:
        st.info("No channels found for the selected filters.")
    else:
        chart_data = g.set_index("CHANNEL_NAME")["SUBSCRIBER_COUNT"]
        st.bar_chart(chart_data)
        st.dataframe(
            g[
                [
                    "CHANNEL_NAME",
                    "CATEGORY_NAME",
                    "COUNTRY_CODE",
                    "SUBSCRIBER_COUNT",
                    "VIEW_COUNT",
                    "GROWTH_LABEL",
                    "LOAD_DATE",
                ]
            ],
            use_container_width=True,
        )

st.markdown("---")

# ===================================================
# 2) Efficiency Scatter – Subscribers vs Views per Video
# ===================================================

st.subheader("2. Channel Efficiency – Subscribers vs Views per Video")

col4, col5 = st.columns(2)

with col4:
    eff_cat = st.selectbox(
        "Filter by Category",
        options=["All"] + sorted(eff_df["CATEGORY_NAME"].dropna().unique().tolist()),
        key="eff_category",
    )

with col5:
    eff_country = st.selectbox(
        "Filter by Country",
        options=["All"] + sorted(eff_df["COUNTRY_CODE"].dropna().unique().tolist()),
        key="eff_country",
    )

e = eff_df.copy()
e = e[(e["SUBSCRIBER_COUNT"] > 0) & (e["VIEWS_PER_VIDEO"] > 0)]

if eff_cat != "All":
    e = e[e["CATEGORY_NAME"] == eff_cat]

if eff_country != "All":
    e = e[e["COUNTRY_CODE"] == eff_country]

if e.empty:
    st.info("No channels found for efficiency plot with current filters.")
else:
    # Limit number of points for plotting sanity
    e_sample = e.head(300).copy()
    scatter_data = (
        e_sample[["CHANNEL_NAME", "SUBSCRIBER_COUNT", "VIEWS_PER_VIDEO"]]
        .set_index("CHANNEL_NAME")
    )

    st.scatter_chart(
        scatter_data,
        x="SUBSCRIBER_COUNT",
        y="VIEWS_PER_VIDEO",
    )

    st.caption(
        "Each point is a channel. Higher VIEWS_PER_VIDEO for a given subscriber "
        "count indicates more efficient content."
    )

    st.dataframe(
        e.sort_values("VPV_EFFICIENCY_INDEX", ascending=False).head(20)[
            [
                "CHANNEL_NAME",
                "CATEGORY_NAME",
                "COUNTRY_CODE",
                "SUBSCRIBER_COUNT",
                "VIEWS_PER_VIDEO",
                "VPV_EFFICIENCY_INDEX",
                "VPS_EFFICIENCY_INDEX",
            ]
        ],
        use_container_width=True,
    )

st.markdown("---")


In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd

st.subheader("Channel Size Distribution – Tiny vs Giants")

session = get_active_session()

# ---------------------------------------------
# Refresh button & cached loader
# ---------------------------------------------
def load_efficiency_df() -> pd.DataFrame:
    if "eff_df" not in st.session_state:
        df = session.sql("""
            SELECT LOAD_DATE,
                   CHANNEL_NAME,
                   SUBSCRIBER_COUNT,
                   VIEW_COUNT
            FROM DB_TEAM_VRR.GOLD.CHANNEL_EFFICIENCY_LEADERBOARD
        """).to_pandas()
        st.session_state["eff_df"] = df
    return st.session_state["eff_df"]

# Button to clear cache so next load hits Snowflake again
if st.button("🔄 Refresh data from Snowflake"):
    st.session_state.pop("eff_df", None)
    st.success("Data cache cleared. Latest data will be loaded from Snowflake on this run.")

# ---------------------------------------------
# Load Gold efficiency table
# ---------------------------------------------
eff_df = load_efficiency_df()

# Ensure LOAD_DATE is full datetime (keep time, no .dt.date)
eff_df["LOAD_DATE"] = pd.to_datetime(eff_df["LOAD_DATE"])

# ---------------------------------------------
# Select snapshot (date + time)
# ---------------------------------------------
available_snapshots = sorted(eff_df["LOAD_DATE"].dropna().unique())

if len(available_snapshots) == 0:
    st.info("No LOAD_DATE values found in CHANNEL_EFFICIENCY_LEADERBOARD.")
else:
    selected_snapshot = st.selectbox(
        "Load Date / Time",
        options=available_snapshots,
        index=len(available_snapshots) - 1,  # default: latest snapshot
        format_func=lambda x: x.strftime("%Y-%m-%d %H:%M:%S"),
        key="size_dist_load_datetime",
    )

    # Filter to selected snapshot
    snap = eff_df[eff_df["LOAD_DATE"] == selected_snapshot].copy()

    # Drop null subs just in case
    snap = snap[snap["SUBSCRIBER_COUNT"].notna()]

    if snap.empty:
        st.info("No subscriber data available for the selected load date/time.")
    else:
        # ---------------------------------------------
        # Bucket channels by subscriber count
        # ---------------------------------------------
        bins = [0, 100_000, 1_000_000, 10_000_000, float("inf")]
        labels = [
            "< 100K (Emerging)",
            "100K–1M (Growing)",
            "1M–10M (Established)",
            "> 10M (Mega)"
        ]

        snap["SIZE_BUCKET"] = pd.cut(
            snap["SUBSCRIBER_COUNT"],
            bins=bins,
            labels=labels,
            right=False
        )

        bucket_stats = (
            snap.groupby("SIZE_BUCKET")
            .agg(
                CHANNEL_COUNT=("CHANNEL_NAME", "count"),
                AVG_SUBS=("SUBSCRIBER_COUNT", "mean")
            )
            .reset_index()
            .sort_values("CHANNEL_COUNT", ascending=False)
        )

        st.write(
            "Selected snapshot: **{}**".format(
                selected_snapshot.strftime("%Y-%m-%d %H:%M:%S")
            )
        )

        # Summary table
        st.dataframe(bucket_stats, use_container_width=True)

        # ---------------------------------------------
        # Donut chart of channel counts by size bucket
        # ---------------------------------------------
        st.vega_lite_chart(
            bucket_stats,
            {
                "mark": {"type": "arc", "innerRadius": 60},
                "encoding": {
                    "theta": {"field": "CHANNEL_COUNT", "type": "quantitative"},
                    "color": {
                        "field": "SIZE_BUCKET",
                        "type": "nominal",
                        "title": "Channel Size",
                        "legend": {"orient": "right"},
                    },
                    "tooltip": [
                        {"field": "SIZE_BUCKET", "type": "nominal", "title": "Bucket"},
                        {
                            "field": "CHANNEL_COUNT",
                            "type": "quantitative",
                            "title": "Number of Channels",
                        },
                        {
                            "field": "AVG_SUBS",
                            "type": "quantitative",
                            "title": "Avg Subscribers",
                            "format": ",.0f",
                        },
                    ],
                },
            },
            use_container_width=True,
        )

        st.caption(
            "Channels are grouped into size buckets based on subscriber count using "
            "GOLD.CHANNEL_EFFICIENCY_LEADERBOARD. This shows how skewed the ecosystem is "
            "toward a small number of very large channels."
        )
